<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import psycopg2
import requests
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [96]:
# текст запроса
query_3_1 = f'''select count(name) as kol_vacanisii
                from vacancies
            '''

In [97]:
# результат запроса
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\3241158491.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,kol_vacanisii
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [98]:
# текст запроса
query_3_2 = f'''select count(id) as kol_rabotadatel
                from employers
            '''

In [99]:
# результат запроса
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\1374563366.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,kol_rabotadatel
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [100]:
# текст запроса
query_3_3 = f'''select count(id) as kol_region
                from areas
            '''

In [101]:
# результат запроса
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\339661163.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,kol_region
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [102]:
# текст запроса
query_3_4 = f'''select count(id) as kol_sferdeyatel
                from industries

            '''

In [103]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\1211709877.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,kol_sferdeyatel
0,294


***

In [104]:
# выводы по предварительному анализу данных


# Выводы по предварительному анализу данных.
- В среднем на одного работодателя приходится 2 вакансии.
- Кол-во регионов в которых расположены работодатели почтив 2 раза меньше чем представлено в соответствующей таблице.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [105]:
# текст запроса
query_4_1 = f'''select count(v.id) as cnt, ar.name as name
                from vacancies as v
                join areas as ar on v.area_id = ar.id
                group by ar.name
                order by cnt desc
            '''

In [106]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\317581882.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,cnt,name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [107]:
# текст запроса
query_4_2 = f'''select count(v.id) as cnt
                from vacancies as v
                where v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
            '''

In [108]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\3981930701.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [109]:
# текст запроса
query_4_3 = f'''select round(avg(salary_from)) as avg_salary_from,
                        round(avg(salary_to)) as avg_salary_to
                from vacancies
            '''

In [110]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\844822074.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [111]:
# текст запроса
query_4_4 = f'''select schedule, employment, count(id) as cnt
                from vacancies
                group by schedule, employment
                order by cnt desc
            '''

In [112]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
df


C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\3686553058.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [113]:
# текст запроса
query_4_5 = f'''select experience
                from vacancies
                where experience is not null
                group by experience
                order by count(id)
            '''

In [114]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\775316018.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

In [115]:
# выводы по детальному анализу вакансий

- Большая часть работодателей зарегистрирована в Москве и Санкт-Петербурге
- Основная часть вакансий с графиком полный день и полная занятость
- Работодатели ищут опытных сотрудников с большим опытом работы

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [116]:
# текст запроса
query_5_1 = f'''select count(v.id) as kol_vacanisii,
                        e.name as name
                from vacancies as v
                left join employers as e on e.id = v.employer_id
                group by e.name
                order by count(v.id) desc
            '''

In [117]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\2991075485.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,kol_vacanisii,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
...,...,...
14761,1,им. Тьюринга
14762,1,LLC Novalab Tech
14763,1,Компания Мир упаковки
14764,1,Лифинцев Олег Иванович


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [118]:
# текст запроса
query_5_2 = f'''select
                        ar.name as name,
                        count(v.id) as cnt_vacanisii,
                        count(e.id) as cnt_employment
                from areas as ar
                left join vacancies as v on ar.id = v.area_id
                left join employers as e on ar.id = e.area
                group by ar.name
                having count(v.id) = 0
                order by cnt_employment desc
            '''

In [119]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\3808390961.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,cnt_vacanisii,cnt_employment
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Ростовская область,0,18
...,...,...,...
588,Струнино,0,0
589,Славгород (Алтайский край),0,0
590,Сорочинск,0,0
591,Ленинский (Тульская область),0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [120]:
# текст запроса
query_5_3 = f'''select
                        e.name as name,
                        count(distinct v.area_id) as cnt_areavacisii
                from employers as e
                left join vacancies as v on e.id = v.employer_id
                group by e.name
                order by cnt_areavacisii desc
            '''

In [121]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\1445531420.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,name,cnt_areavacisii
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [122]:
# текст запроса
query_5_4 = f'''select
                count(*) as cnt
                from employers as e
                left join employers_industries as e_i on e.id = e_i.employer_id
                where e_i.industry_id is null
            '''

In [123]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\273944648.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [124]:
# текст запроса
query_5_5 = f'''select
                e.name as name,
                count(distinct e_i.industry_id) as cnt_industries
                from employers as e
                left join employers_industries as e_i on e.id = e_i.employer_id
                group by e.name
                having count(distinct e_i.industry_id) = 4
            '''

In [125]:
# результат запроса
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\2766408522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name,cnt_industries
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1127,ЮРИОН,4
1128,ЮТИП Технологии,4
1129,ЯКласс,4
1130,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [126]:
# текст запроса
query_5_6 = f'''select
                i.name as name_ind,
                count(*) as cnt
                from employers as e
                left join employers_industries as e_i on e.id = e_i.employer_id
                left join industries as i on e_i.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
                group by i.name
            '''

In [127]:
# результат запроса
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\3602736628.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,name_ind,cnt
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [153]:
# код для получения списка городов-милионников
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
mln_cities = tuple(pd.read_html(url)[0]['Город'])
print(mln_cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [154]:
# текст запроса
query_5_7 = f'''
    select
    a.name as region,
    count(*) as vacancies_count
    from vacancies as v
    join employers as e on v.employer_id = e.id
    join areas as a on v.area_id = a.id
    group by v.area_id, a.name, e.id, e.name
    having a.name in {tuple(mln_cities)} and e.name = 'Яндекс'

    union all

    select 'Total', sum(x.cnt)
    from (
        select count(*) as cnt
        from
        vacancies as v
        join employers as e on v.employer_id = e.id
        join areas as a on v.area_id = a.id
        group by v.area_id, a.name, e.id, e.name
        having a.name in {tuple(mln_cities)} and e.name = 'Яндекс'
        ) as x
    order by 2
    '''

In [155]:
# результат запроса
df = pd.read_sql_query (query_5_7, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_13572\1562245253.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query (query_5_7, connection)


,region,vacancies_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Ростов-на-Дону,25.0
5,Казань,25.0
6,Пермь,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

In [132]:
# выводы по анализу работодателей

Можно отметить, что большинство работодателей ищут работников сразу в нескольких регионах. Больше всего работников ищет "Яндекс" в 181 регионе 485 ваканский приходится на города миллионники.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [8]:
# текст запроса
query_6_1 = f'''select distinct count(v.id)
                from vacancies as v
                where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
            '''

In [9]:
# результат запроса
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\495031047.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [24]:
# текст запроса
query_6_2 = f'''select count(v.id)
                from vacancies as v
                where
                (
                lower(v.name) like '%data scientist%' or
                lower(v.name) like '%data science%' or
                lower(v.name) like '%исследователь данных%' or
                (lower(v.name) like '%ml%' and lower(v.name) not like '%html%') or
                lower(v.name) like '%machine learning%' or
                lower(v.name) like '%машинн%обучен%'
                )
                and
                (
                lower(v.name) like '%junior%' or
                v.experience = 'Нет опыта' or
                v.employment = 'Стажировка'
                )

            '''

In [25]:
# результат запроса
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\2943149581.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [28]:
# текст запроса
query_6_3 = f'''select count(v.id)
                from vacancies as v
                where
                (
                lower(v.name) like '%data scientist%' or
                lower(v.name) like '%data science%' or
                lower(v.name) like '%исследователь данных%' or
                (lower(v.name) like '%ml%' and lower(v.name) not like '%html%') or
                lower(v.name) like '%machine learning%' or
                lower(v.name) like '%машинн%обучен%'
                )
                and
                (
                v.key_skills like '%SQL%' or
                v.key_skills like '%postgres%'
                )
            '''

In [29]:
# результат запроса
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\104054043.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [30]:
# текст запроса
query_6_4 = f'''select count(v.id)
                from vacancies as v
                where
                (
                lower(v.name) like '%data scientist%' or
                lower(v.name) like '%data science%' or
                lower(v.name) like '%исследователь данных%' or
                (lower(v.name) like '%ml%' and lower(v.name) not like '%html%') or
                lower(v.name) like '%machine learning%' or
                lower(v.name) like '%машинн%обучен%'
                )
                and
                v.key_skills like '%Python%'
            '''


In [31]:
# результат запроса
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\1927997822.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [50]:
# текст запроса
query_6_5 = f'''select
                round(avg(length(v.key_skills)-length(replace(v.key_skills, CHR(9), ''))+1), 2) as avg_skill
                from vacancies as v
                where
                (
                v.name ilike '%data scientist%'or
                v.name ilike '%data science%'or
                v.name ilike '%исследователь данных%'or
                (v.name like '%ML%' and v.name not ilike '%HTML%') or
                v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%'
                )
                '''

In [51]:
# результат запроса
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\4273863405.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,avg_skill
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [52]:
# текст запроса
query_6_6 = f'''select
                experience as "опыт работы",
                round(avg(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)), 0) as "средний размер зарплаты"
                from vacancies as v
                where
                (
                v.name ilike '%data scientist%'or
                v.name ilike '%data science%'or
                v.name ilike '%исследователь данных%'or
                (v.name like '%ML%' and v.name not ilike '%HTML%') or
                v.name ilike '%machine learning%' or v.name ilike '%машинн%обучен%'
                )
                and experience = 'От 3 до 6 лет'
                and (salary_from is not null or salary_to is not null)
                group by experience
            '''

In [53]:
# результат запроса
df = pd.read_sql_query (query_6_6, connection)
df

C:\Users\mkondrashovm.DESKTOP-9A3HGE3\AppData\Local\Temp\ipykernel_7732\1432455486.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query (query_6_6, connection)


,опыт работы,средний размер зарплаты
0,От 3 до 6 лет,243115.0


***

In [145]:
# выводы по предметному анализу

# Общий вывод по проекту

- Анализ данных показал, что с помощью этой базы данных можно получать как количественные, так и качественные характеристики различных типов данных, включая информацию о вакансиях, работодателях, регионах и отраслях. Связь между данными в разных таблицах через идентификаторы (ID) позволяет выявлять различные взаимосвязи между ними. На примере вакансий и работодателей можно определить характерные закономерности и тенденции, а также, используя данные о вакансиях для дата-сайентистов, продемонстрировать возможность выделения специфических и узкоспециализированных выборок.

 - Рынок труда в крупных городах, таких как Москва, Санкт-Петербургг, демонстрирует наибольшее количество вакансий. Это связано с концентрацией IT-компаний и стартапов, а также с наличием крупных заказчиков, которые требуют специалистов для работы с большими данными и аналитикой. Меньшие города и регионы, хотя и имеют вакансии, но их количество гораздо ниже, что может свидетельствовать о менее развитой инфраструктуре и меньшем спросе на высококвалифицированных специалистов в таких областях.

- Можно также провести анализ популярности различных ключевых навыков в вакансиях, например, какие языки программирования или технологии востребованы на данный момент в разных областях.

- Интересным будет также исследование перехода вакансий в определённые категории (например, с Junior на Middle или Senior) и как это связано с развитием компании или отрасли.
